# LHC Machine Learing workshop challenge
https://gitlab.cern.ch/IML-WG/IML_challenge_2018/wikis/home

Task: Regress the soft-drop mass of jets with high transverse momentum. Jets are complex physical objects, often containing a spray of particles.https://en.wikipedia.org/wiki/Jet_(particle_physics)


STEP IV

In [1]:
from keras import regularizers,optimizers
from keras.layers import Reshape,Flatten,Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,ELU,LeakyReLU,Dropout,BatchNormalization,concatenate
from keras.models import Model,load_model
from keras import backend as K
from keras import callbacks


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Scroll down for metric evaluation

In [4]:
#get the data
import numpy as np

my_array=np.load('../data/qcd.npy',encoding='bytes')

my_rec_array=my_array.view(np.recarray)


In [68]:
fields=my_array.dtype.names
num_jets=my_rec_array.shape[0]
for field in fields:
    print(field)
    x=getattr(my_rec_array,field)
    x=np.reshape(x,[1,num_jets])
    try:
        my_train_val_array=np.concatenate((my_train_val_array,x))
    except:
        print(field)
        my_train_val_array=x
my_train_val_array=my_train_val_array.T

from sklearn.model_selection import train_test_split

train_array, val_array = train_test_split(my_train_val_array, test_size=0.30,random_state=2158)
train_array.shape,val_array.shape

genjet_sd_m
genjet_sd_m
recojet_pt
recojet_eta
recojet_phi
recojet_m
recojet_sd_pt
recojet_sd_eta
recojet_sd_phi
recojet_sd_m
n_constituents
constituents_pt
constituents_eta
constituents_phi
constituents_charge
constituents_dxy
constituents_dz
constituents_Eem
constituents_Ehad


((729516, 18), (312651, 18))

In [69]:
reco_features_train=train_array[:,0:10] #recofeatures+target
#features1[:3]
#const_features_train=train_array[:,9:] #constituents
reco_features_val=val_array[:,0:10] #recofeatures+target
#features1[:3]
#const_features_val=val_array[:,9:]

In [6]:

train_consts=np.load('train_consts.npz')
train_consts.shape
val_consts=np.load('val_consts.npz')#!!!!!!change it
val_consts.shape

(3000, 1, 8, 130)

In [ ]:
encoder=load_model('my_encoder2_1058716.48974_93.h5')
train_features=encoder.predict(train_consts)
val_features=encoder.predict(train_consts)
train_features.shape

In [70]:
train_target=reco_features_train[0:3000,0]
val_target=reco_features_val[0:3000,0]
reco_features_train=reco_features_train[0:3000,1:]
reco_features_val=reco_features_train[0:3000,1:]#!!!!!!change it
reco_features_train.shape

(3000, 9)

In [ ]:
train_final=np.hstack((reco_features_train,train_features))
val_final=np.hstack((reco_features_val,val_features))
train_final.shape


Load the train regressor

In [119]:
trained_model=load_model('regressor_trial.h5')

In [120]:
train_pred=trained_model.predict(train_final)
print(train_pred.shape)
#val_pred=trained_model.predict(val_final)
train_pred=np.reshape(train_pred,(3000))

(3000, 1)


In [94]:
train_target.shape==train_pred.shape


True

In [116]:
train_pred/train_target

array([1.3294244116103928, 1.2538944687621985, 0.9687673668180001, ...,
       0.0, 0.0, 1.0866181416307288], dtype=object)

Custom loss metric

In [75]:
def evaluate_loss(predictions, truth):  
     ratio = predictions/truth
     a = np.percentile(ratio, 84, interpolation='nearest')  
     b = np.percentile(ratio, 16, interpolation='nearest')  
     c = np.percentile(ratio, 50, interpolation='nearest')  
     loss = (a-b)/(2.*c)  
     return loss
from sklearn.metrics import mean_squared_error

In [113]:
evaluate_loss(train_pred,train_target)

1.0353009270994566

Do the same for test cases, and prep file for submission

In [ ]:
fields=my_array_test.dtype.names
num_jets=my_rec_array_test.shape[0]
for field in fields:
    print(field)
    x=getattr(my_rec_array_test,field)
    x=np.reshape(x,[1,num_jets])
    try:
        my_test_val_array=np.concatenate((my_test_val_array,x))
    except:
        print(field)
        my_test_val_array=x

test_array=my_test_val_array.T
test_consts=np.load('test_consts_reg.npz')['arr_0']
test_features=encoder.predict(test_consts)
reco_features_test=reco_features_test[:,:]

test_final=np.hstack((reco_features_test,test_features))

test_pred=trained_model.predict(test_final)

test_pred=np.reshape(test_pred,(test_pred.shape[0]))

np.save('regression_target_test3',test_pred) #file for submission


